<a href="https://colab.research.google.com/github/shrutin567/Earth-Observation/blob/main/rf_classification_landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Random Forest Supervised Classification

In [ ]:
#import earth engine
import ee

#trigger the authentication flow
ee.Authenticate()

#initialize the library
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Rjm3pBI7lr3BKtn1WhdctDtZYsRaGqg7Kp-ZJF_xXiE&tc=ZQ63xxrptNwT-gc-8eBalIEZroIkIPDfQt_zEN981ho&cc=RcX1m9b4BMJCpYslPE2oGvG2XLAa_l6vtGriBTDqfKo

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BVNt5a0oJW1SLq7dab31p9ca_i2bdDZ8J9HPmEIfxNwKgVL1c7gLRM

Successfully saved authorization token.


In [ ]:
#random forest supervised classification with Landsat
#ROI
roi=ee.Geometry.Point([-122.26,37.87])

#Load Landsat 8 data
image=ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
.filterDate('2017-01-01','2017-12-31')\
.filterBounds(roi)\
.sort('CLOUD_COVER')\
.first()

visParamsTrue={'bands':['B4','B3','B2'],'min':0, 'max':3000, 'gamma':1.4}

#import training data
training=ee.FeatureCollection('users/midekisa/Train_Cover_CA')

label='Class'
bands=['B1','B2','B3','B4','B5','B7']
input=image.select(bands)

#overlay the points on the imagery to get training.
trainImage=input.sampleRegions(**{
    'collection':training,
    'properties':[label],
    'scale':30
})

trainingData=trainImage.randomColumn()
trainSet=trainingData.filter(ee.Filter.lessThan('random',0.8))
testSet=trainingData.filter(ee.Filter.greaterThanOrEquals('random',0.8))

#classification model
#make a random forest classifier and train it
classifier=ee.Classifier.smileRandomForest(10).train(**{
    'features':trainSet,
    'classProperty':label,
    'inputProperties':bands
})

#classify the image
classified=input.classify(classifier);
#print(classified.getInfo());

#define a palette for the classification
landcoverPalette=[
    '#0c2c84',#water(0)
    '#e31a1c',#urban(1)
    '#005a32',#forest(2)
    '#FF8000',#crop(3)
    '#969696',#barren(4)
]

In [ ]:
#import the folium library
import folium

#define a method for displaying ee image tiles to folium map
def add_ee_layer(self,ee_image_object,vis_params,name):
  map_id_dict=ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

#add ee drawing method to folium
folium.Map.add_ee_layer=add_ee_layer

#create a folium map object
my_map=folium.Map(location=[38,-124],zoom_start=8)

#add the layer to the map object
my_map.add_ee_layer(image,visParamsTrue,"Landsat 2017")
my_map.add_ee_layer(classified,{'palette':landcoverPalette,'min':0,'max':4}, 'classification')

#add a layer control panel to the map
my_map.add_child(folium.LayerControl())

#display the map
display(my_map)